In [2]:
# 모듈 임포트
import pandas as pd
import numpy as np
import glob
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# 컬럼명 정의
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current', 'label']

# 정상 데이터
normal_dir = '../realtime/normal/'
normal_files = glob.glob(os.path.join(normal_dir, '*.csv'))
normal_dfs = [pd.read_csv(file, names=column_names, header=None) for file in normal_files]
normal_data = pd.concat(normal_dfs, ignore_index=True)

# 아크 데이터
arc_dir = '../realtime/arc/'
arc_files = glob.glob(os.path.join(arc_dir, '*.csv'))
arc_dfs = [pd.read_csv(file, names=column_names, header=None) for file in arc_files]
arc_data = pd.concat(arc_dfs, ignore_index=True)

# 전체 병합 및 셔플
train_df = pd.concat([normal_data, arc_data], ignore_index=True)
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [4]:
# 파생 피처
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

train_df = add_features(train_df).dropna()

In [5]:
# 슬라이딩 윈도우로 시퀀스 생성
def create_sequences(data, labels, seq_len=8):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(labels[i+seq_len - 1])  # 마지막 시점의 label 사용
    return np.array(X), np.array(y)

features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']
X_raw = train_df[features].values
y_raw = train_df['label'].values.astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

seq_len = 8
X_seq, y_seq = create_sequences(X_scaled, y_raw, seq_len=seq_len)

# CNN 입력 형태로 reshape: (samples, timesteps, features)
X_seq = X_seq.reshape((X_seq.shape[0], seq_len, len(features)))

In [6]:
# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq)


In [7]:
# 모델 정의
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=(seq_len, len(features))),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(128, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\NGN Lab\Desktop\kys\논문\전기안전공학\source\arc-detection-project\powerenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# 학습
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop]
)

Epoch 1/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8724 - loss: 0.3852 - val_accuracy: 0.8856 - val_loss: 0.3481
Epoch 2/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8839 - loss: 0.3557 - val_accuracy: 0.8856 - val_loss: 0.3408
Epoch 3/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8868 - loss: 0.3418 - val_accuracy: 0.8856 - val_loss: 0.3350
Epoch 4/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8877 - loss: 0.3319 - val_accuracy: 0.8854 - val_loss: 0.3297
Epoch 5/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8879 - loss: 0.3243 - val_accuracy: 0.8899 - val_loss: 0.3154
Epoch 6/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8942 - loss: 0.3130 - val_accuracy: 0.8960 - val_loss: 0.3105
Epoch 7/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8916 - loss: 0.3201 - val_accuracy: 0.9000 - val_loss: 0.3046
Epoch 8/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8947 - loss: 0.3089 - val_accuracy: 0.

In [9]:
# 저장
model.save('./model/cnn1d_model.h5')
joblib.dump(scaler, './model/scaler_cnn.joblib')

['./model/scaler_cnn.joblib']

In [10]:
# %%
# TFLite 경량화 및 저장
import tensorflow as tf

# GPU 관련 충돌 방지 (필수 아님, 하지만 권장)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# 모델 로드
model = tf.keras.models.load_model('./model/cnn1d_model.h5')

# 변환기 설정
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # 기본 최적화 (dynamic range quantization)

# 만약 float16 quantization도 시도하고 싶다면 아래 주석을 해제하세요:
# converter.target_spec.supported_types = [tf.float16]

# 변환 수행
tflite_model = converter.convert()

# 저장
with open('./model/cnn1d_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TFLite 변환 및 경량화 완료!")


INFO:tensorflow:Assets written to: C:\Users\NGNLAB~1\AppData\Local\Temp\tmp7gh5ndzu\assets


INFO:tensorflow:Assets written to: C:\Users\NGNLAB~1\AppData\Local\Temp\tmp7gh5ndzu\assets


Saved artifact at 'C:\Users\NGNLAB~1\AppData\Local\Temp\tmp7gh5ndzu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 8, 8), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1927348353936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1927349771664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1927349769936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1927349771280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1927349770320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1927349771088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1927349770512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1927349772816: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ TFLite 변환 및 경량화 완료!
